# Julia Workshop, Day 3: Types and Performance

Goals for today:

- Understand type stability and type groundedness
- Understand how type stability/groundedness allows high performance code from Julia
- Learn tools for dealing with type stability/groundedness
- Learn tools for measuring performance

## Problem 1

Iterator over the prime numbers less than or equal to $n$

In [1]:
import Base: iterate

struct Primes
    n::Int
    sieve::Vector{Bool} # or BitVector
end

In [2]:
function findnexttrue(i, sieve)
    for j in (i+1):length(sieve)
        if sieve[j]
            return j
        end
    end
    nothing
end

# or: findnexttrue(i, sieve) = findfirst(identity, sieve[i:end])

findnexttrue (generic function with 1 method)

In [3]:
# outer constructor
function Primes(n)
    sieve = ones(Bool, n)
    sieve[1] = false # 1 is not prime
    
    p = 1
    while true
        p = findnexttrue(p, sieve)
        if isnothing(p)
            break
        end
        
        # p is a prime, so all multiples of it are not prime
        for i in (2p):p:n
            sieve[i] = false
        end
    end
    
    Primes(n, sieve)
end

Primes

In [4]:
function iterate(p::Primes, state::Int=1)
    next_state = findnexttrue(state, p.sieve)
    if isnothing(next_state)
        nothing
    else
        next_state, next_state
    end
end

iterate (generic function with 248 methods)

In [5]:
primes = Primes(20)
for prime in primes
    print(prime, ", ")
end

2, 3, 5, 7, 11, 13, 17, 19, 

## Problem 2

Outer product matrix type storing O(n+m) elements instead of O(nm).

In [6]:
import Base: getindex, size, adjoint

# type representing uv'
struct OuterProduct{T} <: AbstractMatrix{T}
    u::Vector{T}
    v::Vector{T}
end

getindex(M::OuterProduct, i, j) = M.u[i] * conj(M.v[j])
size(M::OuterProduct) = length(M.u), length(M.v)
adjoint(M::OuterProduct) = OuterProduct(M.v, M.u)

adjoint (generic function with 38 methods)

In [7]:
M = OuterProduct(rand(4), rand(2))

4×2 OuterProduct{Float64}:
 0.10525     0.145235
 0.19789     0.273071
 0.00193939  0.00267619
 0.197672    0.27277

In [8]:
x = rand(2)
M * x

4-element Vector{Float64}:
 0.14814581046585878
 0.27854373654669534
 0.002729824904811223
 0.2782360742385251

## Problem 3

Peano arithmetic

In [9]:
import Base: +, *, convert

abstract type PeanoNumber <: Integer end

function convert(::Type{PeanoNumber}, x::Integer)
    if x < 0
        throw(DomainError(x, "Peano numbers are nonnegative."))
    elseif x == 0
        Zero()
    else
        S(convert(PeanoNumber, x - 1))
    end
end;

In [10]:
struct Zero <: PeanoNumber end # no fields!
struct S{P <: PeanoNumber} <: PeanoNumber
    of::P
end

In [11]:
convert(::Type{Int}, ::Zero) = 0
convert(::Type{Int}, s::S)   = 1 + convert(Int, s.of)

convert (generic function with 196 methods)

In [12]:
+(x::PeanoNumber, ::Zero) = x
+(::Zero, x::PeanoNumber) = x
+(::Zero, ::Zero)         = Zero()
+(x::S, y::S)             = S(+(x, y.of))

+ (generic function with 193 methods)

In [13]:
two = convert(PeanoNumber, 2)
three = convert(PeanoNumber, 3)

S{S{S{Zero}}}(S{S{Zero}}(S{Zero}(Zero())))

In [14]:
two + three

S{S{S{S{S{Zero}}}}}(S{S{S{S{Zero}}}}(S{S{S{Zero}}}(S{S{Zero}}(S{Zero}(Zero())))))

In [15]:
convert(Int, two + three)

5

In [16]:
*(::PeanoNumber, ::Zero) = Zero()
*(::Zero, ::PeanoNumber) = Zero()
*(::Zero, ::Zero)        = Zero()
*(x::S, y::S)            = x + (x * y.of)

* (generic function with 237 methods)

In [17]:
convert(Int, two * three)

6

## What is a type? What is a type system?

- Some types are very intuitive
  - If we have 64 bits in memory, what CPU instruction should we emit for `+`?
  - Types like `UInt64` and `Float64` just assign meaning to some bits
- What about your `Zero <: PeanoNumber` type?

In [18]:
x = 5
println(typeof(x), " takes up ", sizeof(x), " bytes.")

Int64 takes up 8 bytes.


In [19]:
z = Zero()
println(typeof(z), " takes up ", sizeof(z), " bytes.")

Zero takes up 0 bytes.


- Different type systems have different goals
  - A core common theme is that type systems help make sure that your program has _meaning_
  - Type systems attempt to prevent nonsensical operations by making them _type errors_

- To pick on Javascript a little:

```javascript
1 + '1' == '11'
1 - '1' == 0
[] + [] == ''
```

- These are nonsensical operations which produce nonsensical results

- A type system assigns a property called _type_ to every value
  - Either ahead of type (static) or while the program is running (dynamic), check that your program is _well-typed_
  - If not, cause a type error
  - This process is called _type checking_
- Other goals type systems _may_ have:
  - Abstraction
  - Proving things about your code
    - A program in Rust being well-typed means you have constructed a proof that it is memory-safe
  - Allowing a compiler to optimize your code

- This doesn't explain our `Zero` though: what are we assigning a type _to_?
- Programming languages describe a program for an abstract computer
  - It's the job of the compiler and/or interpreter to map between this abstract computer and actual hardware
  - Values in this abstract computer don't always map trivially to bits
- Examples:
  - If a programming language has floating-point numbers, you can do floating point operations whether or not the hardware directly supports them
  - Python and Julia describe programs for an abstract machine with infinite memory

## Julia's Compiler

- Colloquially, a language being compiled usually means that you have a separate step which takes source code and emits machine code which can be directly fed to the CPU
- Technically, compilation is just translation of one machine language to another
  - CPython is both compiled and interpreted
- Julia is compiled, but not _ahead of time_ like C++
  - It's not quite _just in time_ in the sense that tracing JITs like V8 or Numba
  - The term _just ahead of time_ is frequently used
- More specifically: Julia compiles a function whenever it's called with a specific combination of types

In [20]:
foo(x, y, z) = (x + y) / z
foo(1, 2, 3)    # here, we compile a version foo(::Int, ::Int, ::Int)
foo(2, 3, 4)    # here, no compilation happens
foo(1.0, 2, 3); # here, we compile a version foo(::Float64, ::Int, ::Int)

- Core principle making Julia fast:
  - If the compiler can infer all the types and they're all concrete, it can generate the same machine code you could get with C/C++/FORTRAN
  - Automatically determining all the types is called _type inference_
  - If type inference is able to determine the output type of your function using only the input types, your function is _type-stable_
  - If type inference is able to determine the types of every variable in your function, your function is _type-grounded_
- So, to make your Julia program fast:
  1. Pick the right algorithm (!)
  2. Make sure type inference succeeds

- Code goes through several stages of compilation

In [21]:
blah(x, y, z) = (x + y) / z;

In [22]:
@code_lowered blah(1, 2, 3)

CodeInfo(
1 ─ %1 = x + y
│   %2 = %1 / z
└──      return %2
)

In [23]:
@code_llvm blah(1, 2, 3)

;  @ In[21]:1 within `blah`
define double @julia_blah_1679(i64 signext %0, i64 signext %1, i64 signext %2) #0 {
top:
; ┌ @ int.jl:87 within `+`
   %3 = add i64 %1, %0
; └
; ┌ @ int.jl:97 within `/`
; │┌ @ float.jl:294 within `float`
; ││┌ @ float.jl:268 within `AbstractFloat`
; │││┌ @ float.jl:159 within `Float64`
      %4 = sitofp i64 %3 to double
      %5 = sitofp i64 %2 to double
; │└└└
; │ @ int.jl:97 within `/` @ float.jl:412
   %6 = fdiv double %4, %5
   ret double %6
; └
}


In [24]:
@code_native blah(1, 2, 3)

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 14, 0
	.globl	_julia_blah_1716                ; -- Begin function julia_blah_1716
	.p2align	2
_julia_blah_1716:                       ; @julia_blah_1716
; ┌ @ In[21]:1 within `blah`
; %bb.0:                                ; %top
; │┌ @ int.jl:87 within `+`
	add	x8, x1, x0
; │└
; │┌ @ int.jl:97 within `/`
; ││┌ @ float.jl:294 within `float`
; │││┌ @ float.jl:268 within `AbstractFloat`
; ││││┌ @ float.jl:159 within `Float64`
	scvtf	d0, x8
	scvtf	d1, x2
; ││└└└
; ││ @ int.jl:97 within `/` @ float.jl:412
	fdiv	d0, d0, d1
	ret
; └└
                                        ; -- End function
.subsections_via_symbols


## Checking Type Inference

Introducing your new best friend, `@code_warntype`!

- Yes, there is a lot of output
- Remember when you first saw an exception

In [25]:
function mysum(xs::Vector{T}) where {T <: Number}
    ssf = 0
    for x in xs
        ssf += x
    end
    ssf
end

mysum (generic function with 1 method)

In [26]:
@code_warntype mysum([1, 2, 3, 4, 5])

MethodInstance for mysum(::Vector{Int64})
  from mysum(xs::Vector{T}) where T<:Number @ Main In[25]:1
Static Parameters
  T = Int64
Arguments
  #self#::Core.Const(mysum)
  xs::Vector{Int64}
Locals
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  ssf::Int64
  x::Int64
Body::Int64
1 ─       (ssf = 0)
│   %2  = xs::Vector{Int64}
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Int64, Int64}
│         (x = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         (ssf = ssf + x)
│         (@_3 = Base.iterate(%2, %9))
│   %12 = (@_3 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return ssf



In [27]:
@code_warntype mysum([1.0, 2.0, 3.0, 4.0, 5.0])

MethodInstance for mysum(::Vector{Float64})
  from mysum(xs::Vector{T}) where T<:Number @ Main In[25]:1
Static Parameters
  T = Float64
Arguments
  #self#::Core.Const(mysum)
  xs::Vector{Float64}
Locals
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  ssf::Union{Float64, Int64}
  x::Float64
Body::Union{Float64, Int64}
1 ─       (ssf = 0)
│   %2  = xs::Vector{Float64}
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Float64, Int64}
│         (x = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         (ssf = ssf + x)
│         (@_3 = Base.iterate(%2, %9))
│   %12 = (@_3 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return ssf



- `Union` means "one of"

In [28]:
println(typeof(mysum(Float64[])))
println(typeof(mysum([1.0, 2.0])))

Int64
Float64


In [29]:
println(typeof(sum(Float64[])))
println(typeof(sum([1.0, 2.0])))

Float64
Float64


In [30]:
function mybettersum(xs::Vector{T}) where {T <: Number}
    ssf = zero(T)
    for x in xs
        ssf += x
    end
    ssf
end

@code_warntype mybettersum(Float64[])

MethodInstance for mybettersum(::Vector{Float64})
  from mybettersum(xs::Vector{T}) where T<:Number @ Main In[30]:1
Static Parameters
  T = Float64
Arguments
  #self#::Core.Const(mybettersum)
  xs::Vector{Float64}
Locals
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  ssf::Float64
  x::Float64
Body::Float64
1 ─       (ssf = Main.zero($(Expr(:static_parameter, 1))))
│   %2  = xs::Vector{Float64}
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Float64, Int64}
│         (x = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         (ssf = ssf + x)
│         (@_3 = Base.iterate(%2, %9))
│   %12 = (@_3 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return ssf



## On `Union`

- Don't want to give the impression that `Union` is bad
  - Iteration uses it
  - Julia's support for missing values also does
    - A possibly missing value of type `T` is a `Union{T, Missing}`

- These unions are unions of "simple" types
  - `isbitstype`: basically, types which are singletons or map easily to bits
  - A `Union` of a small number of `isbitstype` types is basically stored as `(which, value)`
  - Method lookup turns into essentially an `if/else`
- Some types which are _not_ bitstypes:
  - `Array`, `String`, anything declared with `mutable struct`
    - Then, the value is _boxed_ just like if you have an abstract type

From best (fastest) to worst (slowest):

1. Concrete type
2. Union of all `isbitstype` types
3. Union of non-`isbitstype` types
4. Abstract type

In [31]:
# what's wrong here?
struct Coordinate
    x::AbstractFloat
    y::AbstractFloat
end

import Base: +, -, *

+(a::Coordinate, b::Coordinate) = Coordinate(a.x + b.x, a.y + b.y)
-(a::Coordinate, b::Coordinate) = Coordinate(a.x - b.x, a.y - b.y)
*(λ::Real, a::Coordinate) = Coordinate(λ * a.x, λ * a.y)

* (generic function with 238 methods)

In [42]:
function +(a::Coordinate, b::Coordinate)
    new_x = a.x + b.x
    new_y = a.y + b.y
    Coordinate(new_x, new_y)
end
@code_warntype Coordinate(1.0, 2.0)

MethodInstance for Coordinate(::Float64, ::Float64)
  from Coordinate(x::AbstractFloat, y::AbstractFloat) @ Main In[31]:3
Arguments
  #ctor-self#::Core.Const(Coordinate)
  x::Float64
  y::Float64
Body::Coordinate
1 ─ %1 = %new(Main.Coordinate, x, y)::Core.PartialStruct(Coordinate, Any[Float64, Float64])
└──      return %1



In [33]:
@code_warntype Coordinate(1.0, 2.0f0) + Coordinate(2.0, 4.0f0)

MethodInstance for +(::Coordinate, ::Coordinate)
  from +(a::Coordinate, b::Coordinate) @ Main In[32]:1
Arguments
  #self#::Core.Const(+)
  a::Coordinate
  b::Coordinate
Locals
  new_y::Any
  new_x::Any
Body::Coordinate
1 ─ %1 = Base.getproperty(a, :x)::AbstractFloat
│   %2 = Base.getproperty(b, :x)::AbstractFloat
│        (new_x = %1 + %2)
│   %4 = Base.getproperty(a, :y)::AbstractFloat
│   %5 = Base.getproperty(b, :y)::AbstractFloat
│        (new_y = %4 + %5)
│   %7 = Main.Coordinate(new_x, new_y)::Coordinate
└──      return %7



- Abstract types mean something different in function signatures than they do in structs
  - In functions, we'll compile a separate version for every combination of subtypes
  - We have to explicitly opt in for structs

In [34]:
struct BetterCoordinate{Tx <: AbstractFloat, Ty <: AbstractFloat}
    x::Tx
    y::Ty
end

+(a::BetterCoordinate, b::BetterCoordinate) = BetterCoordinate(a.x + b.x, a.y + b.y)
-(a::BetterCoordinate, b::BetterCoordinate) = BetterCoordinate(a.x - b.x, a.y - b.y)
*(λ::Real, a::BetterCoordinate) = BetterCoordinate(λ * a.x, λ * a.y)

* (generic function with 239 methods)

In [35]:
@code_warntype BetterCoordinate(1.0, 2.0f0) + BetterCoordinate(2.0, 4.0f0)

MethodInstance for +(::BetterCoordinate{Float64, Float32}, ::BetterCoordinate{Float64, Float32})
  from +(a::BetterCoordinate, b::BetterCoordinate) @ Main In[34]:6
Arguments
  #self#::Core.Const(+)
  a::BetterCoordinate{Float64, Float32}
  b::BetterCoordinate{Float64, Float32}
Body::BetterCoordinate{Float64, Float32}
1 ─ %1 = Base.getproperty(a, :x)::Float64
│   %2 = Base.getproperty(b, :x)::Float64
│   %3 = (%1 + %2)::Float64
│   %4 = Base.getproperty(a, :y)::Float32
│   %5 = Base.getproperty(b, :y)::Float32
│   %6 = (%4 + %5)::Float32
│   %7 = Main.BetterCoordinate(%3, %6)::BetterCoordinate{Float64, Float32}
└──      return %7



## What if you really need dynamic types?

- Sometimes you actually need a weird union or an abstract type
  - Maybe someone gave you a very stupid CSV file and fixing it up isn't an option
  - E.g. some field is either a float or a string
- In this case, we want to use a _function barrier_

In [36]:
process(s::String)  = length(s)
process(x::Float64) = convert(Int, trunc(x))
const DATALEN = 4096

4096

In [37]:
function slow_process()
    T = if rand(Bool) # pretend we're checking the first item or something
        String
    else
        Float64
    end
    
    data = Union{String, Float64}[]
    
    for i in 1:DATALEN
        # pretend we're reading
        if T == String
            push!(data, rand(["hello", "goodbye"]))
        else
            push!(data, randn())
        end
    end
    
    data, process.(data)
end

slow_process (generic function with 1 method)

In [38]:
function process_strings()
    data = String[]
    for i in 1:DATALEN
        push!(data, rand(["hello", "goodbye"]))
    end
    data, process.(data)
end

function process_floats()
    data = Float64[]
    for i in 1:DATALEN
        push!(data, randn())
    end
    data, process.(data)
end

function faster_process()
    if rand(Bool)
        process_strings()
    else
        process_floats()
    end
end

faster_process (generic function with 1 method)

In [39]:
using BenchmarkTools
@btime slow_process();

  29.583 μs (4106 allocations: 218.20 KiB)


In [40]:
@btime faster_process();

  17.583 μs (11 allocations: 154.25 KiB)


## Putting `@code_warntype` to use

- juliaworkshop.miakramer.space
- Download the file labeled "`@code_warntype` and type stability problems"
- There are several small problems
- The first couple may or may not be obvious, but I would highly recommend using `@code_warntype` for practice regardless
  - Several of the types are matrices and vectors. Try multiplication, getindex, etc.